In [1]:
import re
import json
import pandas as pd

In [2]:
with open('../data/TCMSV_alldesc.json', encoding='utf-8') as f:
    data = json.load(f)
data

{'data': {'UPDATETIME': 'Wed Dec 21 00:00:00 CST 2022',
  'park': [{'id': '001',
    'area': '信義區',
    'name': '府前廣場地下停車場',
    'type': '1',
    'type2': '2',
    'summary': '為地下二層停車場，計有1998個小型車停車格，1337個機車停車位',
    'address': '松壽路1號地下',
    'tel': '2722-0152',
    'payex': '小型車全日月票4200元，夜間月票1000元(限周一至周五19-8，及周六、日與行政機關放假之紀念日、民俗日)，小型車計時30元(9-18)，夜間計時10元(18-9)；機車計時10元(當日當次上限20元)，機車月票300元。111/11/26、11/27、12/3、12/4、12/10、12/11、12/17、12/18、12/24、12/25、12/31、112/1/1、1/2、1/8、1/14、1/15，10-20時，調整小型車費率，計時60元。',
    'serviceTime': '00:00:00~23:59:59',
    'tw97x': '306812.928',
    'tw97y': '2769892.95',
    'totalcar': 2043,
    'totalmotor': 1360,
    'totalbike': 0,
    'totalbus': 0,
    'Pregnancy_First': '40',
    'Handicap_First': '45',
    'Taxi_OneHR_Free': '0',
    'AED_Equipment': '0',
    'CellSignal_Enhancement': '0',
    'Accessibility_Elevator': '0',
    'Phone_Charge': '0',
    'Child_Pickup_Area': '0',
    'FareInfo': {'WorkingDay': [{'Period': '00~09', 'Fare': '10'},
      {'Per

In [3]:
index = []
area = []
name = []
payex = []
summary = []
address = []
totalcar = []
totalmotor = []
FareInfo_workday = []
FareInfo_holiday = []
twd97_x = []
twd97_y = []

count = 0

length = len(data['data']['park'])
for i in range(length):
    index.append(data['data']['park'][i]['id'])
    area.append(data['data']['park'][i]['area'])
    name.append(data['data']['park'][i]['name'])
    payex.append(data['data']['park'][i]['payex'])
    summary.append(data['data']['park'][i]['summary'])
    address.append(data['data']['park'][i]['address'])
    totalcar.append(data['data']['park'][i]['totalcar'])
    totalmotor.append(data['data']['park'][i]['totalmotor'])
    if bool(data['data']['park'][i]['FareInfo']):
        FareInfo_workday.append(data['data']['park'][i]['FareInfo']['WorkingDay'])
        FareInfo_holiday.append(data['data']['park'][i]['FareInfo']['Holiday'])
    else:
        FareInfo_workday.append([{'Period': "00~24", 'Fare': "None"}])
        FareInfo_holiday.append([{'Period': "00~24", 'Fare': "None"}])
        count += 1
    twd97_x.append(float(data['data']['park'][i]['tw97x']))
    twd97_y.append(float(data['data']['park'][i]['tw97y']))

In [4]:
parktype = []
for txt in summary:
    if '平面' in txt:
        parktype.append('平面式')
    elif '立體' in txt:
        parktype.append('立體式')
    elif '塔台' in txt:
        parktype.append('塔台式')
    elif '機械' in txt:
        parktype.append('機械式')
    else:
        parktype.append('其他')

In [5]:
# for i in range(1, 25):
#     exec('W{} = []'.format(i))
#     exec('H{} = []'.format(i))
maxFare_workday = []
maxFare_holiday = []

for fare_info in FareInfo_workday:
    fareList = []
    for info in fare_info:
        # print(info['Period'], info['Fare'])
        if info['Fare'] != 'None':
            fareList.append(int(info['Fare']))
        else:
            fareList.append(-9)
    maxFare_workday.append(max(fareList))

# for fare_info in FareInfo_holiday:
#     fareList = []
#     for info in fare_info:
#         print(info['Period'], info['Fare'])
#         if type(info['Fare']) != str:
#             fareList.append(int(info['Fare']))
#         else:
#             fareList.append(-9)
#     maxFare_holiday.append(max(fareList))

In [6]:
cnt = 0
fareinfo = []
for i in range(len(payex)):
    txt = payex[i]
    txt_split = re.split('，|、|。', txt)
    result = []
    for j in range(len(txt_split)):
        h0 = re.findall(r'^計時\d*元', txt_split[j])
        h1 = re.findall(r'\D*計時\d*元', txt_split[j])
        h2 = re.findall(r'\D*\S*\d*元/時', txt_split[j])
        h3 = re.findall(r'每小時\d+元', txt_split[j])
        if len(h0) > 0:
            result.append('h0:' + h0[0])
        elif len(h1) > 0:
            result.append('h1:' + h1[0])
        elif len(h2) > 0:
            result.append('h2:' + h2[0])
        elif len(h3) > 0:
            result.append('h3:' + h3[0])
    if len(result) > 0:
        # print(len(result))
        fareinfo.append(int(re.findall(r'\d+', result[0])[1]))
    else:
        cnt += 1
        fareinfo.append(-9)
    print(i, result)

0 ['h1:小型車計時30元', 'h1:夜間計時10元', 'h0:計時60元']
1 ['h2:計時：30元/時', 'h2:40元/時', 'h2:10元/時', 'h0:計時60元']
2 ['h1:小型車計時30元', 'h1:) 機車計時10元']
3 ['h1:周一至周四計時40元']
4 ['h1:小型車計時30元']
5 []
6 ['h1:小型車計時20元']
7 ['h2:計時：小型車平日20元/時', 'h2:10元/時', 'h2:平面場收費上限80元；假日30元/時', 'h2:40元/時', 'h2:10元/時']
8 ['h2:計時：小型車/大型重型機車30元/時']
9 ['h2:80元/時']
10 ['h2:計時:80元/時']
11 ['h2:計時:80元/時']
12 ['h2:120元/時']
13 ['h0:計時30元']
14 ['h2:計時：小型車周一至周五(10-22)50元/時', 'h2:(22-10)10元/時', 'h2:周日與行政機關放假之紀念日與民俗日(10-24)60元/時', 'h2:(24-10)10元/時']
15 ['h1:計時：周一至周五計時40元', 'h0:計時10元', 'h1:周六日及政府行政機關放假之紀念日與民俗節日計時40元', 'h0:計時10元']
16 ['h2:120元/時']
17 ['h1:)計時20元', 'h1:)計時10元', 'h1:)計時20元', 'h1:)計時30元', 'h1:)計時10元']
18 ['h0:計時30元']
19 []
20 ['h1:)計時20元', 'h1:)計時40元']
21 ['h1:周一至周四計時40元', 'h1:周五至周日及政府行政機關放假之紀念日及民俗日計時50元']
22 ['h2:計時：小型車30元/時', 'h2:10元/時', 'h2:大型重型機車30元/時', 'h2:10元/時']
23 ['h1:)計時50元', 'h1:)計時10元']
24 ['h0:計時30元']
25 ['h2:計時：100元/時']
26 ['h0:計時20元']
27 ['h1:)計時10元', 'h1:)計時30元', 'h1:)計時10元', 'h1:)計時60元']
28 ['h1:小型車計時40元', 'h0:計時

In [7]:
# d = {
#     'id': index, 'area': area, 'name': name, 'summary': summary, 'address': address, 
#     'totalcar': totalcar, 'totalmotor': totalmotor, 'FareInfo_workday': FareInfo_workday, 'FareInfo_holiday': FareInfo_holiday, 
#     'twd97x': twd97_x, 'twd97y': twd97_y
# }

d = {
    'id': index, 'area': area, 'name': name, 'type': parktype, 'payex': payex, 'summary': summary, 'address': address, 
    'totalcar': totalcar, 'totalmotor': totalmotor, 'FareInfo': fareinfo, 
    'twd97x': twd97_x, 'twd97y': twd97_y
}

df = pd.DataFrame(data=d, index=range(length))
df

,id,area,name,type,payex,summary,address,totalcar,totalmotor,FareInfo,twd97x,twd97y
0,001,信義區,府前廣場地下停車場,其他,小型車全日月票4200元，夜間月票1000元(限周一至周五19-8，及周六、日與行政機關放假...,為地下二層停車場，計有1998個小型車停車格，1337個機車停車位,松壽路1號地下,2043,1360,30,306812.92800,2.769893e+06
1,002,信義區,松壽廣場地下停車場,立體式,計時：30元/時(周一至周五9-22)、40元/時(周六至周日與行政機關放假之紀念日與民俗日...,立體式小型車446格(含身心障礙停車位6格)。,松智路15號地下,455,0,30,307143.74000,2.770012e+06
2,003,信義區,臺北市災害應變中心地下停車場,立體式,小型車計時30元，全日月票5000元，日間月票2600元(7-19) 機車計時10元(當日當...,立體式小型車169格(含身心障礙停車位5格)。立體式機車208格(含身心障礙停車位5格)。,莊敬路391巷11弄2號地下,169,200,30,307142.12100,2.769088e+06
3,004,信義區,雅祥公園地下停車場,立體式,周一至周四計時40元 周五至周日及政府行政機關放假之紀念日與民俗日計時50元 全日月票480...,立體式小型車204格(含身心障礙停車位5格)。,松隆路123巷7號地下,204,0,40,307703.05900,2.771155e+06
4,005,北投區,立農公園地下停車場,立體式,小型車計時30元，全日月票4800元，日間月票2800元(7-19)，夜間月票2000元(1...,立體式小型車204格(含身心障礙停車位5格)。立體式機車95格(含身心障礙停車位2格)。,承德路7段372號地下,204,95,30,300781.89200,2.778990e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
1848,MA77,大安區,馥裕和平停車場,平面式,"月租：2,200元/月。",一般平面式小型車32格(含身心障礙停車位1格),,32,0,-9,306703.00358,2.767279e+06
1849,MA81,中正區,TIMES延平開封,平面式,計時：100元/時，停車未滿1小時以1小時計，逾1小時以上者，未滿半小時以半小時計,一般平面式小型車10格(含身心障礙停車位1格),,10,0,100,301539.58143,2.771041e+06
1850,MA86,士林區,華展停車場,立體式,計時:小型車100元/時，停車未滿1小時以1小時計，逾1小時以上者，未滿半小時以半小時計，車...,建物附設立體式小型車120格(含身心障礙停車位3格)；建物附設立體式機車60格(含身心障礙停...,,120,0,100,303578.51998,2.778938e+06
1851,MA87,內湖區,江南停車場,立體式,60元/時，全程以半小時計費。,建物附設立體式小型車79格(含身心障礙停車位2格),,79,0,60,308287.10786,2.774489e+06


In [58]:
others = []
for i in range(len(payex)):
    txt = payex[i]
    x1 = re.findall(r'\D+小型車\D+\d+元/時', txt)
    x2 = re.findall(r'小型車計時\d+元', txt)
    x3 = re.findall(r'計時\D*\d+元', txt)
    h1 = re.findall(r'計時', txt)
    h2 = re.findall(r'計時:70元/時', txt)
    x4 = re.findall(r'\d+元\D*\d*\D*\d*\D*\D*.*時', txt)
    x5 = re.findall(r'每小時\d+元', txt)
    e1 = re.findall(r'計次\D?\d+元', txt)
    e2 = re.findall(r'\D*\d+\D*次', txt)
    x8 = re.findall(r'日間月票\d+元', txt)
    x9 = re.findall(r'夜間月票\d+元', txt)
    x10 = re.findall(r'全日月票\d+元', txt)
    x11 = re.findall(r'全日\d+\D*\d+元/月', txt)
    m1 = re.findall(r'全天\d*\D*\d*元/月', txt)
    x12 = re.findall(r'夜間\d+\D*\d+元/月', txt)
    x13 = re.findall(r'月租\D*\d+\D*\d+元\D*', txt)
    x14 = re.findall(r'\d*\D*\d*\D*月\d*\D*\d*\D*', txt)
    ct1 = re.findall(r'\D*汽車\d+元', txt)
    o = re.findall(r'\D+公告', txt)
    s1 = re.findall(r'\d*\D*\d+\D*季', txt)
    s2 = re.findall(r'每季\d*\D*\d*元', txt)
    s3 = re.findall(r'季票\d*\D*\d+元', txt)
    s4 = re.findall(r'季繳\d*\D*\d+元', txt)
    h1 = re.findall(r'半年繳\d*\D*\d+元', txt)
    h2 = re.findall(r'\d*\D*\d*元.*半年\D?', txt)
    f = re.findall('免費', txt)
    b = re.findall('', txt)
    # x4 = re.findall(r'計次')
    if len(x1) > 0:
        print(x1)
    elif len(x2) > 0:
        print(x2)
    elif len(x3) > 0:
        print(x3)
    elif len(h1) > 0:
        print(h1)
    # elif len(h2) > 0:
    #     print(h2)
    elif len(x4) > 0:
        print(x4)
    elif len(x5) > 0:
        print(x5)
    elif len(e1) > 0:
        print(e1)
    elif len(e2) > 0:
        print(e2)
    elif len(x8) > 0:
        print('日間月票', x8)
    elif len(x9) > 0:
        print('夜間月票', x9)
    elif len(x10) > 0:
        print('全日月票', x10)
    elif len(x11) > 0:
        print('全日月票', x11)
    elif len(m1) > 0:
        print('全日月票', m1)
    elif len(x12) > 0:
        print('夜間月票', x12)
    elif len(x13) > 0:
        print('全日月票', x13)
    elif len(x14) > 0:
        print('全日月票', x14)
    elif len(ct1) > 0:
        print(ct1)
    elif len(o) > 0:
        print(o)
    elif len(s1) > 0:
        print(s1)
    elif len(s2) > 0:
        print(s2)
    elif len(s3) > 0:
        print(s3)
    elif len(s4) > 0:
        print(s4)
    elif len(h1) > 0:
        print(h1)
    elif len(h2) > 0:
        print(h2)
    elif len(f) > 0:
        print('免費')
    elif len(b) > 0:
        print('無資訊')
    else:
        print(i, '其他', txt)
        others.append(i)

['小型車計時30元']
['計時：30元', '計時60元']
['小型車計時30元']
['計時40元', '計時50元']
['小型車計時30元']
['20元/次(隔日另計，以凌晨2時']
['小型車計時20元']
['計時：小型車平日20元/時']
['計時：小型車/大型重型機車30元/時']
['80元/時，停車未滿1小時以1小時計，逾1小時以上，未滿半小時以半小時']
['計時:80元']
['計時:80元']
['120元/時，停車未滿1小時以1小時計，逾1小時以上者，未滿半小時以半小時']
['計時30元']
['50元/時、(22-10)10元/時；小型車周六、周日與行政機關放假之紀念日與民俗日(10-24)60元/時、(24-10)10元/時，全程以半小時']
['計時：周一至周五計時40元', '計時10元', '計時40元', '計時10元']
['120元/時，停車未滿1小時以1小時計，逾1小時以上者，未滿半小時以半小時']
['計時20元', '計時10元', '計時20元', '計時30元', '計時10元']
['計時30元']
日間月票 ['日間月票1500元']
['計時20元', '計時40元']
['計時40元', '計時50元']
['計時：小型車30元']
['計時50元', '計時10元']
['計時30元']
['計時：100元']
['計時20元']
['計時10元', '計時30元', '計時10元', '計時60元']
['小型車計時40元']
['計時：周一至五40元']
['計時：50元']
['計時30元', '計時10元']
['計時：20元']
['計時20元', '計時10元']
['計時40元', '計時30元']
['計時：40元']
['計時30元', '計時10元']
['計時20元', '計時30元', '計時30元']
['計時20元']
['計時：30元']
['小型車計時40元']
['計時40元', '計時10元', '計時10元']
['計時：小型車20元']
['計時30元']
['計時40元']
['計時：小型車平日20元/時']
['計時：100元']
['計時40元', '計時10元', '計時50元', '計時10元', '計時60元']
['計時：30元']
['70

In [59]:
others

[]

In [60]:
len(others)

0

In [151]:
cnt = 0
fareinfo = []
for i in range(len(payex)):
    txt = payex[i]
    txt_split = re.split('，|、|。', txt)
    result = []
    for j in range(len(txt_split)):
        h0 = re.findall(r'^計時\d*元', txt_split[j])
        h1 = re.findall(r'\D*計時\d*元', txt_split[j])
        h2 = re.findall(r'\D*\S*\d*元/時', txt_split[j])
        h3 = re.findall(r'每小時\d+元', txt_split[j])
        if len(h0) > 0:
            result.append('h0:' + h0[0])
        elif len(h1) > 0:
            result.append('h1:' + h1[0])
        elif len(h2) > 0:
            result.append('h2:' + h2[0])
        elif len(h3) > 0:
            result.append('h3:' + h3[0])
    if len(result) > 0:
        # print(len(result))
        fareinfo.append(int(re.findall(r'\d+', result[0])[1]))
    else:
        cnt += 1
        fareinfo.append(-9)
    print(i, result)

0 ['h1:小型車計時30元', 'h1:夜間計時10元', 'h0:計時60元']
1 ['h2:計時：30元/時', 'h2:40元/時', 'h2:10元/時', 'h0:計時60元']
2 ['h1:小型車計時30元', 'h1:) 機車計時10元']
3 ['h1:周一至周四計時40元']
4 ['h1:小型車計時30元']
5 []
6 ['h1:小型車計時20元']
7 ['h2:計時：小型車平日20元/時', 'h2:10元/時', 'h2:平面場收費上限80元；假日30元/時', 'h2:40元/時', 'h2:10元/時']
8 ['h2:計時：小型車/大型重型機車30元/時']
9 ['h2:80元/時']
10 ['h2:計時:80元/時']
11 ['h2:計時:80元/時']
12 ['h2:120元/時']
13 ['h0:計時30元']
14 ['h2:計時：小型車周一至周五(10-22)50元/時', 'h2:(22-10)10元/時', 'h2:周日與行政機關放假之紀念日與民俗日(10-24)60元/時', 'h2:(24-10)10元/時']
15 ['h1:計時：周一至周五計時40元', 'h0:計時10元', 'h1:周六日及政府行政機關放假之紀念日與民俗節日計時40元', 'h0:計時10元']
16 ['h2:120元/時']
17 ['h1:)計時20元', 'h1:)計時10元', 'h1:)計時20元', 'h1:)計時30元', 'h1:)計時10元']
18 ['h0:計時30元']
19 []
20 ['h1:)計時20元', 'h1:)計時40元']
21 ['h1:周一至周四計時40元', 'h1:周五至周日及政府行政機關放假之紀念日及民俗日計時50元']
22 ['h2:計時：小型車30元/時', 'h2:10元/時', 'h2:大型重型機車30元/時', 'h2:10元/時']
23 ['h1:)計時50元', 'h1:)計時10元']
24 ['h0:計時30元']
25 ['h2:計時：100元/時']
26 ['h0:計時20元']
27 ['h1:)計時10元', 'h1:)計時30元', 'h1:)計時10元', 'h1:)計時60元']
28 ['h1:小型車計時40元', 'h0:計時

0 ['h8:小型車計時30元', 'h10:夜間計時10元(18-9)；機車計時10元', 'h0:計時60元']
1 ['h11:計時：30元/時', 'h11:40元/時', 'h11:10元/時', 'h0:計時60元']
2 ['h8:小型車計時30元', 'h5:2600元(7-19) 機車計時', 'h5:10元(合計每小時']
3 ['h5:40元 周五至周日及政府行政機關放假之紀念日與民俗日計時']
4 ['h8:小型車計時30元', 'h5:10元(合計每小時']
5 ['詳收費規定']
6 ['h8:小型車計時20元', 'h5:10元(合計每小時']
7 ['h11:計時：小型車平日20元/時', 'h11:10元/時', 'h11:平面場收費上限80元；假日30元/時', 'h11:40元/時', 'h11:10元/時']
8 ['h11:計時：小型車/大型重型機車30元/時']
9 ['h11:80元/時']
10 ['h11:計時:80元/時']
11 ['h11:計時:80元/時']
12 ['h11:120元/時']
13 ['h0:計時30元', 'h5:10元(合計每小時']
14 ['h11:計時：小型車周一至周五(10-22)50元/時', 'h11:(22-10)10元/時', 'h11:周日與行政機關放假之紀念日與民俗日(10-24)60元/時', 'h11:(24-10)10元/時']
15 ['h3:計時：周一至周五計時40元(08-20)', 'h0:計時10元', 'h3:計時40元(10-20)', 'h0:計時10元']
16 ['h11:120元/時']
17 ['h10:周一至周五(8-22)計時20元', 'h10:(22-8)計時10元', 'h10:周六至周日及政府行政機關放假之紀念日及民俗節日(8-10)計時20元', 'h10:(10-22)計時30元', 'h10:(22-8)計時10元', 'h5:10元(合計每小時']
18 ['h0:計時30元', 'h5:10元(合計每小時']
19 ['詳收費規定']
20 ['h10:) (2-19)計時20元', 'h10:(19-2)計時40元', 'h5:10元(合計每小時']
21 ['h7:計時', 'h7:計時', 'h5:10元(合計

In [19]:
df.to_csv('../data/parking_0104.csv', encoding='utf-8', index=False)